In [1]:
import re
import json
import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import csv

In [2]:
def get_ld_json(url: str) -> dict:
    parser = "html.parser"
    req = requests.get(url)
    soup = bs(req.text, parser)
    return json.loads("".join(soup.find("script", {"type":"application/ld+json"}).contents))

def main_feature_extractor(feature):
    feature_soup = soup.find("ul",{"class":'gridded--list bordered--grid'})
    if feature_soup is None:
        feature_soup = feature_soup = soup.find("ul",{"class":'gridded--list'})
    feature = feature_soup.findChild("img",{"alt":feature}).find_parent("li").get_text().strip()
    return(feature)
    

def facility_checker(facility):
    if facility in facilities:
        return(1)
    else:
        return(0)

In [3]:
Apt_Name = [None]*6000
No_Rooms = [None]*6000
Street = [None]*6000
Locality = [None]*6000
Region = [None]*6000
Longitude = [None]*6000
Latitude = [None]*6000
Unit_Name = [None]*6000
Unit_ID = [None]*6000
Bathroom = [None]*6000
Furnished = [None]*6000
Area = [None]*6000
Floor = [None]*6000
Tower = [None]*6000

In [4]:
AC = [None]*6000
Water_Heater = [None]*6000
Dining_Set = [None]*6000
Electricity = [None]*6000
Bed = [None]*6000
Access_Card = [None]*6000
Kitchen = [None]*6000
Fridge = [None]*6000
Washing_Machine = [None]*6000
TV = [None]*6000
ATM = [None]*6000
TV_Cable = [None]*6000
Grocery = [None]*6000
Internet = [None]*6000
Swim_Pool = [None]*6000
Laundry = [None]*6000
Security = [None]*6000
Basketball = [None]*6000
Multipurpose_Room = [None]*6000
Gym = [None]*6000
Jogging = [None]*6000
Tennis= [None]*6000
Restaurant = [None]*6000
Playground = [None]*6000
Total_Facilities = [None]*6000
Currency = [None]*6000
Annual_Price = [None]*6000
Monthly_Price = [None]*6000
Deposit_Currency = [None]*6000
Service_Currency = [None]*6000
Deposit_Charge = [None]*6000
Service_Charge = [None]*6000

In [5]:
with open('apartment_links.csv', 'r') as f:
    URL_list = [row[0] for row in csv.reader(f)] 

URL_list = URL_list[1:]

In [6]:
for i, url in enumerate(URL_list):
    try:
        if i%100 == 0:
            print("Passed iteration {}".format(i))
            
        r = requests.get(url)
            
        soup = bs(r.content)
    
        ##Apartment Metadata
        apartment_metadata = get_ld_json(url)
        Apt_Name[i] = apartment_metadata['name']
    
        No_Rooms[i] = apartment_metadata['numberOfRooms']
    
        Street[i] = apartment_metadata['address']['streetAddress']
    
        Locality[i] = apartment_metadata['address']['addressLocality']
    
        Region[i] = apartment_metadata['address']['addressRegion']
    
        Longitude[i] = apartment_metadata['geo']['longitude']
    
        Latitude[i] = apartment_metadata['geo']['latitude']
    
        h1_title = soup.find("div", {"id": "units"})
        h1_title = h1_title.find('h1')
        unit_name = h1_title.get_text().strip()
        Unit_Name[i] = unit_name
    
        unit_id = url[-7:]
        Unit_ID[i] = unit_id
    
        #Apartment Features
        bathroom = main_feature_extractor('Bathroom')
        number_of_bathrooms = int(bathroom[0])
        Bathroom[i] = number_of_bathrooms
    
        furnish = main_feature_extractor('Furnish')
        if furnish == 'Fully Furnished':
            furnish_status = 1
        else:
            furnish_status = 0
        Furnished[i] = furnish_status    
    
        area_text = main_feature_extractor('Area')
        area = float(area_text[:-2].strip())
        Area[i] = area
    
        Floor[i] = main_feature_extractor('Floor')
    
        Tower[i] = main_feature_extractor('Tower')
    
        ##Apartment Facilities
        facilities_soup = soup.find_all('span', {"class":"facility-text"})
        facilities = str()
        facilities_list = []
        for _ in facilities_soup:
            facility = _.get_text().strip()
            facilities = facilities + facility+' '
            facilities_list.append(facility)

        AC[i] = facility_checker('AC')
    
        Water_Heater[i] = facility_checker('Water Heater')
    
        Dining_Set[i] = facility_checker('Dining Set')
    
        Electricity[i] = facility_checker('Electricity')
    
        Bed[i] = facility_checker('Bed')
    
        Access_Card[i] = facility_checker('Access Card')
    
        Kitchen[i] = facility_checker('Kitchen')
    
        Fridge[i] = facility_checker('Refrigerator')
    
        Washing_Machine[i] = facility_checker('Washing Machine')
        
        TV[i] = facility_checker('TV')
    
        ATM[i] = facility_checker('ATM')
    
        TV_Cable[i] = facility_checker('TV Kabel')
    
        Grocery[i] = facility_checker('Grocery Store')
    
        Internet[i] = facility_checker('Internet')
    
        Swim_Pool[i] = facility_checker('Kolam Renang')
    
        Laundry[i] = facility_checker('Laundry')
    
        Security[i] = facility_checker('Security')
    
        Basketball[i] = facility_checker('Lapangan Basket')
    
        Multipurpose_Room[i] = facility_checker('Ruang Serbaguna')
    
        Gym[i] = facility_checker('Gym')
    
        Jogging[i] = facility_checker('Jogging Track')
    
        Tennis[i] = facility_checker('Lapangan Tenis')
    
        Restaurant[i] = facility_checker('Restoran')
    
        Playground[i] = facility_checker('Taman Bermain')
    
        Total_Facilities[i] = AC[i] + Water_Heater[i] + Dining_Set[i] + Electricity[i] + Bed[i] + Access_Card[i] + \
                            Kitchen[i] + Fridge[i] + Washing_Machine[i] + TV[i] + ATM[i] + TV_Cable[i] + Grocery[i] + \
                            Internet[i] + Swim_Pool[i] + Laundry[i] + Security[i] + Basketball[i] + Multipurpose_Room[i] + \
                            Gym[i] + Jogging[i] + Tennis[i] + Restaurant[i] + Playground[i]
    
        #Apartment Price
        price = soup.find('div', {'class':'price-content'})
        price.find('span',{'class':'text-strikethrough'})
        if price.find('span',{'class':'text-strikethrough'}) is not None:
            price.find('span',{'class':'text-strikethrough'}).decompose()
        price_raw = price.get_text().replace('\n','').replace(' ','').replace('Rp0','').replace('$0','').replace(',','').replace('$','USD').replace('Rp','IDR').strip()
    
        pattern_year = "['USD|IDR']\d+/['tahun'|'thn']"
        pattern_month = "['USD'|'IDR']\d+/['bulan'|'bln']"
        search_year_regex = re.search(pattern_year, price_raw)
        if search_year_regex is not None:
            search_year = search_year_regex[0]
    
        search_month_regex = re.search(pattern_month, price_raw)
        if search_month_regex is not None:
            search_month = search_month_regex[0]
        else:
            search_month = ''
        
        if search_year[0] == 'D':
            currency = 'USD'
        elif search_year[0] == 'R':
            currency = 'IDR'
        else:
            currency = 'unknown'
    
        annual_price = search_year[1:].replace('/t','').strip()

        if search_month == '':
            monthly_price = 'NA'
        else:
            monthly_price = search_month[1:].replace('/b','').strip()

        Currency[i] = currency
        Annual_Price[i] = annual_price
        Monthly_Price[i] = monthly_price
    
    
        monthlyfees = soup.find('div',{'id':'monthlyfees'})
        fee_charges = monthlyfees.get_text().strip().replace('.','').strip().replace('Rp','IDR').replace('$','USD').split('\n')
        service_fee_currency = ''
        service_fee = ''
        deposit_fee_currency = ''
        deposit_fee = ''
        fee_charges
        for _ in fee_charges:    
            if 'Service' in _:
                if 'ree' in _:  #anticipating for free charges
                    service_fee_currency = ''
                    service_fee = ''
                else:
                    pattern = "['IDR'|'USD'] \d+"
                    fee_search = re.search(pattern,_)
                    if fee_search is None:
                        service_fee_currency = ''
                        service_fee = ''
                    else:
                        fee = fee_search[0]
                        if fee[0] == 'D':
                            service_fee_currency= 'USD'
                            service_fee = int(fee.replace('R','').replace('D','').strip())
                        elif fee[0] == 'R':
                            service_fee_currency = 'IDR'
                            service_fee = int(fee.replace('R','').replace('D','').strip())
                        else:
                            service_fee_currency = 'unknown'
                            service_fee = int(fee.replace('R','').replace('D','').strip())
        
            
            elif 'Deposit' in _:
                if 'ree' in _: #anticipating for free charges
                    deposit_fee_currency = ''
                    deposit_fee = ''
                else:
                    pattern = "['IDR'|'USD'] \d+"
                    fee_search = re.search(pattern,_)
                    if fee_search is None:
                        deposit_fee_currency = ''
                        deposit_fee = ''
                    else:
                        fee = fee_search[0]
                        if fee[0] == 'D':
                            deposit_fee_currency= 'USD'
                            deposit_fee = int(fee.replace('R','').replace('D','').strip())
                        elif fee[0] == 'R':
                            deposit_fee_currency = 'IDR'
                            deposit_fee = int(fee.replace('R','').replace('D','').strip())
                        else:
                            deposit_fee_currency = 'unknown'
                            deposit_fee = int(fee.replace('R','').replace('D','').strip())
            
    
        Deposit_Currency[i] = deposit_fee_currency
        Deposit_Charge[i] = deposit_fee
        Service_Currency[i] = service_fee_currency
        Service_Charge[i] = service_fee
    
    except:
        print("ERROR at iteration {}".format(i))
        Apt_Name[i] = 'ERROR'
        No_Rooms[i] = 'ERROR'
        Street[i] = 'ERROR'
        Locality[i] = 'ERROR'
        Region[i] = 'ERROR'
        Longitude[i] = 'ERROR'
        Latitude[i] = 'ERROR'
        Unit_Name[i] = 'ERROR'
        Unit_ID[i] = 'ERROR'
        Bathroom[i] = 'ERROR'
        Furnished[i] = 'ERROR'
        Area[i] = 'ERROR'
        Floor[i] = 'ERROR'
        Tower[i] = 'ERROR'
        AC[i] = 'ERROR'
        Water_Heater[i] = 'ERROR'
        Dining_Set[i] = 'ERROR'
        Electricity[i] = 'ERROR'
        Bed[i] = 'ERROR'
        Access_Card[i] = 'ERROR'
        Kitchen[i] = 'ERROR'
        Fridge[i] = 'ERROR'
        Washing_Machine[i] = 'ERROR'
        TV[i] = 'ERROR'
        ATM[i] = 'ERROR'
        TV_Cable[i] = 'ERROR'
        Grocery[i] = 'ERROR'
        Internet[i] = 'ERROR'
        Swim_Pool[i] = 'ERROR'
        Laundry[i] = 'ERROR'
        Security[i] = 'ERROR'
        Basketball[i] = 'ERROR'
        Multipurpose_Room[i] = 'ERROR'
        Gym[i] = 'ERROR'
        Jogging[i] = 'ERROR'
        Tennis[i] = 'ERROR'
        Restaurant[i] = 'ERROR'
        Playground[i] = 'ERROR'
        Total_Facilities[i] = 'ERROR'
        Currency[i] = 'ERROR'
        Annual_Price[i] = 'ERROR'
        Monthly_Price[i] = 'ERROR'
        Deposit_Currency[i] = 'ERROR'
        Service_Currency[i] = 'ERROR'
        Deposit_Charge[i] = 'ERROR'
        Service_Charge[i] = 'ERROR'
        pass


        

Passed iteration 0
ERROR at iteration 39
Passed iteration 100
Passed iteration 200
ERROR at iteration 223
Passed iteration 300
Passed iteration 400
Passed iteration 500
Passed iteration 600
Passed iteration 700
Passed iteration 800
Passed iteration 900
Passed iteration 1000
ERROR at iteration 1011
Passed iteration 1100
ERROR at iteration 1111
Passed iteration 1200
ERROR at iteration 1207
Passed iteration 1300
Passed iteration 1400
Passed iteration 1500
Passed iteration 1600
ERROR at iteration 1604
Passed iteration 1700
ERROR at iteration 1745
Passed iteration 1800
Passed iteration 1900
Passed iteration 2000
Passed iteration 2100
Passed iteration 2200
Passed iteration 2300
ERROR at iteration 2320
ERROR at iteration 2321
ERROR at iteration 2374
Passed iteration 2400
Passed iteration 2500
ERROR at iteration 2541
Passed iteration 2600
Passed iteration 2700
Passed iteration 2800
Passed iteration 2900
Passed iteration 3000
Passed iteration 3100
Passed iteration 3200
Passed iteration 3300
Pas

In [7]:
import pandas as pd
df = pd.DataFrame(list(zip(URL_list, Unit_Name, Unit_ID, Apt_Name, No_Rooms, Street, Locality, Region, Longitude, Latitude, Furnished, Area, Floor, Tower, AC, Water_Heater, Dining_Set, Electricity, Bed, Access_Card, Kitchen, Fridge, Washing_Machine, TV, ATM, TV_Cable, Grocery, Internet, Swim_Pool, Laundry, Security, Basketball, Multipurpose_Room, Gym, Jogging, Tennis, Restaurant, Playground, Total_Facilities, Currency, Annual_Price, Monthly_Price, Deposit_Currency, Deposit_Charge, Service_Currency, Service_Charge)))
df.columns = ['URL', 'Unit_Name', 'Unit_ID', 'Apt_Name', 'No_Rooms', 'Street', 'Locality', 'Region', 'Longitude', 'Latitude', 'Furnished', 'Area', 'Floor', 'Tower', 'AC', 'Water_Heater', 'Dining_Set', 'Electricity', 'Bed', 'Access_Card', 'Kitchen', 'Fridge', 'Washing_Machine', 'TV', 'ATM', 'TV_Cable', 'Grocery', 'Internet', 'Swim_Pool', 'Laundry', 'Security', 'Basketball', 'Multipurpose_Room', 'Gym', 'Jogging', 'Tennis', 'Restaurant', 'Playground', 'Total_Facilities', 'Currency', 'Annual_Price', 'Monthly_Price', 'Deposit_Currency', 'Deposit_Charge', 'Service_Currency', 'Service_Charge']

In [8]:
df.to_csv('Apartment Dataset Raw.csv')